In [4]:
import os 
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from tqdm import tqdm

load_dotenv()

True

In [3]:
df = pd.read_csv("../data/data_playlists_enem.csv")

In [5]:
df.head()

,titles,content
0,HISTÓRIA GERAL #8 FEUDALISMO,Title: HISTÓRIA GERAL #8 FEUDALISMO\nViews cou...
1,HISTÓRIA GERAL #8 FEUDALISMO,Title: HISTÓRIA GERAL #8 FEUDALISMO\nViews cou...
2,HISTÓRIA GERAL #8 FEUDALISMO,Title: HISTÓRIA GERAL #8 FEUDALISMO\nViews cou...
3,HISTÓRIA GERAL #8 FEUDALISMO,Title: HISTÓRIA GERAL #8 FEUDALISMO\nViews cou...
4,HISTÓRIA GERAL #8 FEUDALISMO,Title: HISTÓRIA GERAL #8 FEUDALISMO\nViews cou...


In [ ]:
# prepare editais 
type_exs =['CUET', 'Exames Nacionais', 'EXANI', 'ICFES', 'SAT']
file_paths = ["../data/editais/" + type_ex + "/edital.pdf" for type_ex in type_exs] 

In [19]:
from langchain_community.document_loaders import PyPDFLoader
from tqdm import tqdm

documents = {}
for file_path in tqdm(file_paths):
    loader = PyPDFLoader(file_path)
    document = loader.load()
    documents[file_path.split("/")[-2]] = document

100%|██████████| 5/5 [00:23<00:00,  4.78s/it]


In [28]:
all_one_shot_strings = {}
for ex in documents:
    one_shot_string = ""
    for page in documents[ex]:
        one_shot_string += page.page_content + "\n"
    all_one_shot_strings[ex] = one_shot_string

In [33]:
# Save txt
for ex in all_one_shot_strings:
    with open(f"../data/editais/{ex}/edital.txt", "w", encoding="utf8") as f:
        f.write(all_one_shot_strings[ex])

In [34]:
# Provas

In [39]:
type_exs_filename =[('SAT', "exame/sat-practice-test-4-digital.pdf")]
file_paths = ["../data/provas/" + x[0] + f"/{x[1]}" for x in type_exs_filename] 

In [44]:
documents = {}
for file_path in tqdm(file_paths):
    loader = PyPDFLoader(file_path)
    document = loader.load()
    documents[file_path.split("/")[-3]] = document

all_one_shot_strings = {}
for ex in documents:
    one_shot_string = ""
    for page in documents[ex]:
        one_shot_string += page.page_content + "\n"
    all_one_shot_strings[ex] = one_shot_string

100%|██████████| 1/1 [00:03<00:00,  3.26s/it]


In [45]:
all_one_shot_strings

{'SAT': 't:J \n~ \n829939\n11111111111111111111 IIIIII \n6TSL05\nThe SAT® \nPractice\nTest# 4\n \nMake time to take the practice test. \nIt is one of the best ways to get ready \nfor the SAT. \nAfter you have taken the practice test, score it \nright away at sat.org/digital-practice. \nThis version of the SAT Practice Test is for students who will be taking \nthe digital SAT in nondigital format. \n11\n\nTest begins on the next page.\nModule \n1 \nReading and Writing \n33 QUESTIONS \nDIRECTIONS \nThe questions in this section address a number of important reading and writing skills. Each \nquestion includes one or more passages, which may include a table or graph. Read each passage \nand question carefully, and then choose the best answer to the question based on the passage(s). \nAll questions in this section are multiple -choice with four answer choices. Each question has a \nsingle best answer. \n---------~ ---------~ \n1 \nThe spacecraft OSIRIS-REx briefly made contact \nwith \nthe

In [46]:
# Save txt
for ex in all_one_shot_strings:
    with open(f"../data/provas/{ex}/exame/prova.txt", "w", encoding="utf8") as f:
        f.write(all_one_shot_strings[ex])

## Build CSV

In [5]:
# prepare editais 
type_exs =['CUET', 'Exames Nacionais', 'EXANI', 'ICFES', 'SAT']
file_paths = ["../data/editais/" + type_ex + "/edital.txt" for type_ex in type_exs] 

documents = {}
for file_path in tqdm(file_paths):
    with open(file_path, "r", encoding="utf8") as f:
        content = f.read()
    documents[file_path.split("/")[-2]] = content

100%|██████████| 5/5 [00:00<00:00, 822.96it/s]


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

output = {}
for ex in type_exs:

    text_splitter = RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size=750,
        chunk_overlap=250,
        length_function=len,
        is_separator_regex=False,
    )
    texts = [x.page_content for x in text_splitter.create_documents([documents[ex]])]
    output[ex] = texts

In [8]:

pd.DataFrame(output["CUET"], columns=["content"]).to_csv("../data/cuet_edital.csv", index=False)
pd.DataFrame(output["Exames Nacionais"], columns=["content"]).to_csv("../data/exames_nacionais_edital.csv", index=False)
pd.DataFrame(output["EXANI"], columns=["content"]).to_csv("../data/exani_edital.csv", index=False)
pd.DataFrame(output["ICFES"], columns=["content"]).to_csv("../data/icfes_edital.csv", index=False)
pd.DataFrame(output["SAT"], columns=["content"]).to_csv("../data/sat_edital.csv", index=False)